In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import numpy as np

In [0]:
from scipy.spatial.distance import cdist

def eval_MAP_K(query_set, search_set, query_label, search_set_label, K):
    query_class_arr = query_set
    search_set_class_arr = search_set

    query_label_arr = query_label
    query_label_arr.astype(int)
    search_set_label_arr = search_set_label
    search_set_label_arr.astype(int)

    num_query_sample = query_label_arr.shape[0]

    dist_q_search_class = cdist(query_class_arr,search_set_class_arr,metric='euclidean')

    AP=0
    for query_count in range(0,num_query_sample):

        actual_label = query_label_arr[query_count]
        value=dist_q_search_class[query_count,:]
        sorted_value_idx=value.argsort()
        predicted_K_label = search_set_label_arr[sorted_value_idx[0:K]]
        L=np.sum((np.in1d(predicted_K_label,actual_label)).astype(int))

        P=0
        for r in range(0,K):
            rtr_label_r = predicted_K_label[:(r+1)]
            l=np.sum((np.in1d(rtr_label_r,actual_label)).astype(int))
            precision=l/(r+1)
            if actual_label==predicted_K_label[r]:
                delta=1
            else:
                delta=0
            P=P+(precision*delta)
        if L!=0:
            AP=AP+((1/L)*P)
    map_K=AP/num_query_sample
    return map_K

In [0]:
import os
import numpy as np
dictionary={}
training_files='/content/gdrive/My Drive/TUB_Feature_Vectors_Balanced/Sketchs/Training'
labels=os.listdir(training_files)
labels.sort()
x_train_skt=[]
y_train=[]
y_t=[]
a=0
for label in labels:
  skt_fea_names=os.listdir(os.path.join(training_files,label))
  for skt in skt_fea_names:
    skt_fea_path=os.path.join(training_files,label,skt)
    skt_fea=np.load(skt_fea_path)
    x_train_skt.append(skt_fea)
    y_t.append(a)
    y_train.append(label)
    a=a+1
x_train_skt=np.asarray(x_train_skt)

In [0]:
from sklearn.preprocessing import OneHotEncoder
ohe_skt=OneHotEncoder()
one_hot_skt=ohe_skt.fit_transform(np.asarray(y_train).reshape(92,64))
one_hot_skt=one_hot_skt.toarray()
one_hot_skt=one_hot_skt.transpose()

In [0]:
img_training_files='/content/gdrive/My Drive/TUB_Feature_Vectors_Balanced/Image/Training'
labels=os.listdir(img_training_files)
labels.sort()
x_train_img=[]
y_train_img=[]
y_t_i=[]
b=0
for label in labels:
  img_names=os.listdir(os.path.join(img_training_files,label))
  for img in img_names:
    img_fea_path=os.path.join(img_training_files,label,img)
    img_fea=np.load(img_fea_path)
    x_train_img.append(img_fea)
    y_t_i.append(b)
    y_train_img.append(label)
    b=b+1
x_train_img=np.asarray(x_train_img)

In [0]:
import pandas as pd
temp=pd.DataFrame(y_train_img)
unique_labels=temp[0].unique()
unique_labels.sort()
a=unique_labels.reshape(92,1)
ohe_img=OneHotEncoder()
one_hot_img=ohe_img.fit_transform(a)
one_hot_img=one_hot_img.toarray()
one_hot_img=one_hot_img.transpose()
f=[]
for row in one_hot_img:
   for i in range(64):
    f.append(row)
f=np.asarray(f)

In [0]:
f[1]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [0]:
imagelabels[1]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [0]:
#-------------------------------encoder-----------------------------------------------------------------

input_img = Input(shape=(4096,))

#layer1_img = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(input_img)

# "encoded" is the encoded representation of the input

#layer2_img = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(layer1_img)


encoded_img = Dense(92, activation='relu',kernel_regularizer=regularizers.l2(0.1))(input_img)


#--------------------------------decoder-------------------------------------------------------------------


#layer3_img = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(encoded_img)

#layer4_img  = Dense(1024, activation = 'relu',kernel_regularizer=regularizers.l2(0.1))(layer3_img)

# "decoded" is the lossy reconstruction of the input

decoded_img = Dense(4096,activation='relu',kernel_regularizer=regularizers.l2(0.1))(encoded_img)



# this model maps an input to its reconstruction
autoencoder_img = Model(input_img, [decoded_img,encoded_img])

In [0]:
encoder_img = Model(input_img, encoded_img)

In [0]:
autoencoder_img.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense_6 (Dense)              (None, 92)                376924    
_________________________________________________________________
dense_7 (Dense)              (None, 4096)              380928    
Total params: 757,852
Trainable params: 757,852
Non-trainable params: 0
_________________________________________________________________


In [0]:
encoder_img.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense_6 (Dense)              (None, 92)                376924    
Total params: 376,924
Trainable params: 376,924
Non-trainable params: 0
_________________________________________________________________


In [0]:
#---------------------------------encoder-----------------------------------------------------

input_skt = Input(shape=(4096,))

# "encoded" is the encoded representation of the input

#layer1_skt = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(input_skt)

#layer2_skt = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(input_skt)


encoded_skt = Dense(92,activation='relu',kernel_regularizer=regularizers.l2(0.1))(input_skt)


#---------------------------------decoder------------------------------------------------------


#layer3_skt = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(encoded_skt)

#layer4_skt = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(layer3_skt)


# "decoded" is the lossy reconstruction of the input
decoded_skt = Dense(4096, activation='relu',kernel_regularizer=regularizers.l2(0.1))(encoded_skt)

# this model maps an input to its reconstruction
autoencoder_skt = Model(input_skt, [decoded_skt,encoded_skt])

In [0]:
encoder_skt = Model(input_skt, encoded_skt)

In [0]:
autoencoder_skt.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense_8 (Dense)              (None, 92)                376924    
_________________________________________________________________
dense_9 (Dense)              (None, 4096)              380928    
Total params: 757,852
Trainable params: 757,852
Non-trainable params: 0
_________________________________________________________________


In [0]:
encoder_skt.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense_8 (Dense)              (None, 92)                376924    
Total params: 376,924
Trainable params: 376,924
Non-trainable params: 0
_________________________________________________________________


In [0]:
parllel = Model([input_skt,input_img],[decoded_skt,encoded_skt,decoded_img,encoded_img])

In [0]:
parllel.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 92)           376924      input_5[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 92)           376924      input_4[0][0]                    
___________________________________________________________________________________________

In [0]:
from tensorflow.keras.optimizers import Adam 
parllel.compile(optimizer=Adam(lr=1e-4),loss=['mean_squared_error','mean_squared_error','mean_squared_error','mean_squared_error'])

In [0]:
parllel.fit([x_train_skt,x_train_img],[x_train_skt,f,x_train_img,f],epochs=150,shuffle=True,batch_size=128,verbose=1)

Epoch 1/150
5888/5888 [==============================] - 1s 107us/sample - loss: 66.6589 - dense_9_loss: 1.6617 - dense_8_loss: 0.2410 - dense_7_loss: 2.1941 - dense_6_loss: 0.3600
Epoch 2/150
5888/5888 [==============================] - 0s 71us/sample - loss: 47.8904 - dense_9_loss: 1.6754 - dense_8_loss: 0.0474 - dense_7_loss: 2.2131 - dense_6_loss: 0.0632
Epoch 3/150
5888/5888 [==============================] - 0s 71us/sample - loss: 34.2218 - dense_9_loss: 1.6749 - dense_8_loss: 0.0345 - dense_7_loss: 2.2133 - dense_6_loss: 0.0461
Epoch 4/150
5888/5888 [==============================] - 0s 77us/sample - loss: 24.4371 - dense_9_loss: 1.6714 - dense_8_loss: 0.0247 - dense_7_loss: 2.2105 - dense_6_loss: 0.0319
Epoch 5/150
5888/5888 [==============================] - 0s 71us/sample - loss: 17.5643 - dense_9_loss: 1.6659 - dense_8_loss: 0.0193 - dense_7_loss: 2.2048 - dense_6_loss: 0.0237
Epoch 6/150
5888/5888 [==============================] - 0s 73us/sample - loss: 12.8250 - dense_9_l

In [0]:
def get_skt_data():
  training_query_skt=[]
  training_query_label=[]
  z_s=[]
  q=0
  z=0
  for skt_fea in x_train_skt:
    z_s.append(skt_fea)
    training_query_skt.append(encoder_skt.predict(skt_fea.reshape(4096,1).transpose()))
    training_query_label.append(q)
    z=z+1
    if z%64 ==0:
      q=q+1
  return np.squeeze(np.asarray(training_query_skt)),np.asarray(training_query_label).reshape(5888,1),z_s


def get_img_data():
  training_search_img=[]
  training_search_img_label=[]
  z_i=[]
  p=0
  z=0
  for img_fea in x_train_img:
    z_i.append(img_fea)
    training_search_img.append(encoder_img.predict(img_fea.reshape(4096,1).transpose()))
    training_search_img_label.append(p)
    z=z+1
    if z%64 == 0:
      p=p+1
  return np.squeeze(np.asarray(training_search_img)),np.asarray(training_search_img_label).reshape(5888,1),z_i


training_query_skt,training_query_label,z_s=get_skt_data()
training_search_img,training_search_img_label,z_i=get_img_data()
eval_MAP_K(training_query_skt,training_search_img,training_query_label,training_search_img_label,200)


0.4590233253247556